In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_1059426/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_4_9_5,0.998093,0.993737,0.994728,0.994329,0.229987,0.527642,0.544008,0.535345,"Hidden Size=[15, 5], regularizer=0.1, learning..."
1,model_4_9_6,0.998088,0.993325,0.994195,0.993860,0.230603,0.562371,0.598936,0.579576,"Hidden Size=[15, 5], regularizer=0.1, learning..."
2,model_4_9_4,0.998074,0.994135,0.995279,0.994801,0.232284,0.494069,0.487146,0.490811,"Hidden Size=[15, 5], regularizer=0.1, learning..."
3,model_4_9_7,0.998065,0.992910,0.993687,0.993403,0.233329,0.597279,0.651408,0.622752,"Hidden Size=[15, 5], regularizer=0.1, learning..."
4,model_4_9_8,0.998030,0.992502,0.993205,0.992962,0.237550,0.631675,0.701131,0.664357,"Hidden Size=[15, 5], regularizer=0.1, learning..."
...,...,...,...,...,...,...,...,...,...,...
1187,model_8_7_4,0.980172,0.978620,0.966760,0.973772,2.391214,1.554715,2.708177,2.097523,"Hidden Size=[20, 15], regularizer=0.1, learnin..."
1196,model_6_9_3,0.980023,0.994684,0.980541,0.987203,2.409228,0.548785,3.174021,1.784186,"Hidden Size=[15, 7], regularizer=0.1, learning..."
1218,model_7_6_3,0.979574,0.979875,0.972566,0.979525,2.463393,2.939650,4.423286,3.637829,"Hidden Size=[20, 15], regularizer=0.01, learni..."
1227,model_1_8_2,0.979357,0.987812,0.990258,0.988845,2.489547,1.963938,1.195345,1.602249,"Hidden Size=[15, 10], regularizer=0.01, learni..."
